In [2]:
import tensorflow as tf

In [3]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34, 31]

In [4]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [5]:
for sales in tf_dataset:
    print(sales)  #tensor object

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


In [6]:
for sales in tf_dataset:
    print(sales.numpy()) #numpy object

21
22
-108
31
-1
32
34
31


In [7]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales) #also numpy object

21
22
-108
31
-1
32
34
31


In [8]:
for sales in tf_dataset.take(3): #Prints first three elements
    print(sales.numpy())

21
22
-108


In [9]:
tf_dataset = tf_dataset.filter(lambda x:x>0) #Eliminating negative values
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
21
22
31
32
34
31


In [10]:
tf_dataset = tf_dataset.map(lambda x:x*72) #for manipulation of all the data
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [11]:
tf_dataset = tf_dataset.shuffle(2) #buffer size = 2, creates window size of 2
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1584
2232
2304
1512
2448
2232


In [12]:
for sales_batch in tf_dataset.batch(3): #divides dataset into batches
    print(sales_batch.numpy())

[1584 1512 2232]
[2304 2232 2448]


In [13]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y:y*72).shuffle(2).batch(2)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1584 2232]
[1512 2304]
[2448 2232]


In [14]:
images_ds = tf.data.Dataset.list_files('PetImages/*/*',shuffle=False)

for file in images_ds.take(3):
    print(file.numpy) #reads images path

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=string, numpy=b'PetImages\\Cat\\0.jpg'>>
<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=string, numpy=b'PetImages\\Cat\\1.jpg'>>
<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=string, numpy=b'PetImages\\Cat\\10.jpg'>>


In [15]:
images_ds = images_ds.shuffle(200)

for file in images_ds.take(3):
    print(file.numpy())

b'PetImages\\Cat\\10152.jpg'
b'PetImages\\Cat\\10177.jpg'
b'PetImages\\Cat\\1013.jpg'


In [16]:
class_names = ["cat","dog"] #specifiying different classes

In [17]:
image_count = len(images_ds)
image_count

18335

In [31]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'PetImages\\Cat\\10073.jpg'
b'PetImages\\Cat\\10066.jpg'
b'PetImages\\Cat\\10201.jpg'


In [18]:
train_size = int(image_count*0.8) # Seggregating 80% of data for training

train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [19]:
len(train_ds)

14668

In [20]:
len(test_ds)

3667

In [32]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep) #Gives the class of the image
    return parts[-2]

In [25]:
def process_image(file_path):
    label = get_label(file_path)
    
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128,128])
    
    return img, label

In [23]:
for t in train_ds.take(4):
    print(t.numpy())

b'PetImages\\Cat\\10081.jpg'
b'PetImages\\Cat\\10020.jpg'
b'PetImages\\Cat\\10018.jpg'
b'PetImages\\Cat\\10169.jpg'


In [28]:
for img, label in train_ds.map(process_image).take(3): #contains label
    print(img)
    print(label)

tf.Tensor(
[[[ 70.39276   63.39276   70.39276 ]
  [ 68.98578   61.98578   68.98578 ]
  [ 68.32135   61.32135   68.32135 ]
  ...
  [ 70.841675  98.841675 122.841675]
  [ 74.84485  102.84485  126.84485 ]
  [ 84.73297  111.73297  132.73297 ]]

 [[ 75.4389    68.4389    75.4389  ]
  [ 71.08563   64.08563   71.08563 ]
  [ 72.        65.        72.      ]
  ...
  [ 68.46796   96.46796  118.46796 ]
  [ 78.309204 106.309204 128.3092  ]
  [ 82.177185 109.177185 130.17719 ]]

 [[ 75.79193   68.79193   75.79193 ]
  [ 72.80859   65.80859   72.80859 ]
  [ 71.540344  64.540344  71.540344]
  ...
  [ 75.65631  103.65631  124.65631 ]
  [ 76.77112  104.77112  125.77112 ]
  [ 81.86041  108.86041  129.86041 ]]

 ...

 [[ 73.497925  93.404175 109.310425]
  [ 71.74304   93.46179  110.82117 ]
  [ 67.55463   91.08588  109.61713 ]
  ...
  [ 82.51422  104.51422  118.51422 ]
  [ 84.25696  106.25696  120.25696 ]
  [ 77.294556 100.56409  114.025024]]

 [[ 77.65375   97.56     113.46625 ]
  [ 75.52344   97.24219  1

In [29]:
def scale(image, label):
    return image/255, label

In [33]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [34]:
for image, label in train_ds.take(1):
    print("***Image: ",image)
    print("***Label: ",label)

***Image:  tf.Tensor(
[[[216.       157.        63.      ]
  [215.46094  156.46094   62.460938]
  [217.56833  158.56833   64.56833 ]
  ...
  [179.3887   165.18558  142.3848  ]
  [167.46399  153.46399  127.46399 ]
  [172.49417  158.49417  132.49417 ]]

 [[215.       156.        62.      ]
  [214.51953  155.51953   61.51953 ]
  [216.50577  157.50577   63.505768]
  ...
  [170.65775  156.57758  133.40793 ]
  [170.2597   156.2597   130.2597  ]
  [173.6272   159.6272   133.6272  ]]

 [[215.69531  156.69531   62.695312]
  [215.69531  156.69531   62.695312]
  [217.69531  158.69531   64.69531 ]
  ...
  [169.1124   155.66528  130.59647 ]
  [167.27289  153.27289  127.27289 ]
  [180.35956  166.35956  140.35956 ]]

 ...

 [[156.11725  119.11725   64.11725 ]
  [148.4295   111.429504  56.429504]
  [153.98047  116.98047   61.98047 ]
  ...
  [186.37885  170.57806  147.9765  ]
  [174.0238   158.0238   135.0238  ]
  [181.25775  165.25775  142.25775 ]]

 [[151.01205  114.012054  59.012054]
  [148.51038  1

In [35]:
train_ds = train_ds.map(scale)

In [36]:
for image, label in train_ds.take(5):
    print("***Image: ",image.numpy()[0][0])
    print("***Label: ",label.numpy())

***Image:  [0.42745098 0.3254902  0.03529412]
***Label:  b'Cat'
***Image:  [0.5028646 0.5028646 0.4636489]
***Label:  b'Cat'
***Image:  [0.29803923 0.3019608  0.2784314 ]
***Label:  b'Cat'
***Image:  [0.32349494 0.3462278  0.48213083]
***Label:  b'Cat'
***Image:  [0.14650735 0.15042892 0.12689951]
***Label:  b'Cat'
